In [8]:
# idk where to put this lol
# also im just messing around here
import inspect
import dill

# Frame definitions and init

In [41]:
class Frame():
    # I, Noemi, refer to this as a FrameNode to remove ambiguity with FrameObjects for the time being. 
    is_global = False

    def __init__(self, name = None, bindings=None, parent=None, fobj=None):
        self.parent = parent
        self.children = set()
        if bindings is None:
            self.bindings = {}
        self.name = name
        # self's frame object
        self.fobj = fobj

    def bind(self, name, value):
        self.bindings[name] = value # Note: this will allow for duplicate entries. need to figure out a better way to do this. 
    
    def set_name(self, name):
        self.name = name
    
    def add_child(self, child):
        self.children.add(child)

In [45]:
# a dictionary containing frames with associated variables in the form of a tuple: (type, name, mem loc, value)
env_frames = {"global":{}}
fobj_name_pairs = {}
# associates a frame object to a frame node (or in the case of global, at least for the time being, "global" to a frame node.)
fobj_framenode_pairs = {}
env_mutables = {}
cnter = 1

# TODO: maybe name "___main___" instead of global? hmmmmmm but if we have wrapper func then no. 
def add_new_frame(name = None, variables = None, frame_dict = env_frames): # func should always refer to the caller
    if variables is None:
        variables = {}
    # if no frame nodes have been initialized, set the frame as the global frame and the root
    if len(fobj_framenode_pairs.keys()) == 0:
        name = "global"
        fobj_framenode_pairs[name] = Frame(bindings = variables) # TODO: add fobj
        # OLD: TODO: also current_fobj not listed, fix
        env_frames[name] = {"name": None, "parent": None, "parent_fobj": None, "current_fobj": None, "variables":variables}

    else: 
        curr_frame = inspect.currentframe().f_back
        name = curr_frame.f_code.co_name
        f_name = "f" + name #temp TODO: add number
        parent_frame = curr_frame.f_back
        # TODO: why global be stupid? fix.
        parent = fobj_framenode_pairs[parent_frame] if parent_frame in fobj_framenode_pairs else fobj_framenode_pairs["global"] # frame_name_pairs[parent_frame] if parent_frame in frame_name_pairs else "global"
        
        # OLD: dictionary rep of frame associations. Use for debugging only. 
        env_frames[f_name] = {"name": name, "parent": parent, "parent_fobj": parent_frame, "curr_fobj": curr_frame, "variables":variables}
        # add self to the pairing dict
        fobj_framenode_pairs[curr_frame] = Frame(name = name, bindings = variables, parent = parent, fobj=curr_frame)
        # modify parent to include self as a child
        parent.add_child(fobj_framenode_pairs[curr_frame])
        # OLD: dictionary rep between fobj and frame
        fobj_name_pairs[curr_frame] = name
    return fobj_framenode_pairs, env_frames, fobj_name_pairs # messy

In [48]:
#print("global", inspect.currentframe())
def glob():
    def A():
        z = 7
        #print("AHHHH")
        def B():
            y = 4
            #print("B", dill.detect.varnames(B), inspect.currentframe().f_back)
            def C():
                x = 5
                #print("C", inspect.currentframe().f_back)
                locs = locals()
                variables = {} #(type(locs[var]), var, dill.pointers.at(locs[var]), locs[var]) for var in locs]
                fobj_framenode_pairs, env_frames, frame_name_pairs = add_new_frame(variables)
                return 8 + x + y + z
            variables = {} #(type(locals()[var]), var, dill.pointers.at(locals()[var]), locals()[var]) for var in locals()]
            fobj_framenode_pairs, env_frames, frame_name_pairs = add_new_frame(variables)
            return C
        def D():
            #print("C2", inspect.currentframe().f_back)
            locs = locals()
            variables = {var:(type(locs[var]), locs[var]) for var in locs}
            fobj_framenode_pairs, env_frames, frame_name_pairs = add_new_frame(variables)
            return B()
        #print("A", inspect.currentframe().f_back, inspect.currentframe())
        locs = locals()
        #variables = [(type(locs[var]), var, dill.pointers.at(locs[var]), locs[var]) for var in locs.keys()]
        returnvar = D()()
        variables = {var:(type(locs[var]), locs[var]) for var in locs} # TODO: check whether output of locs[var] is mutable
        fobj_framenode_pairs, env_frames, frame_name_pairss = add_new_frame(variables) # messy
        print("in A ", inspect.currentframe().f_lineno)
        return returnvar
    cat = A()
#print(fobj_framenode_pairs["fA"].f_lineno)
glob()
print(env_frames["fA"]["curr_fobj"].f_lineno)
env_frames

in A  31
32


{'global': {'name': None,
  'parent': None,
  'parent_fobj': None,
  'current_fobj': None,
  'variables': {}},
 'fB': {'name': 'B',
  'parent': <__main__.Frame at 0x10d8604d0>,
  'parent_fobj': <frame at 0x10d7b20c0, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_38469/2442529524.py', line 24, code D>,
  'curr_fobj': <frame at 0x10d7a6b20, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_38469/2442529524.py', line 18, code B>,
  'variables': {}},
 'fC': {'name': 'C',
  'parent': <__main__.Frame at 0x10d653a10>,
  'parent_fobj': <frame at 0x10d78e340, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_38469/2442529524.py', line 32, code A>,
  'curr_fobj': <frame at 0x10d7a5e00, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_38469/2442529524.py', line 15, code C>,
  'variables': {}},
 'fA': {'name': 'A',
  'parent': <__main__.Frame at 0x10d653a10>,
  'parent_fobj': <frame at 0x10d80bb60, file '/var/folders/8l/7g3122r17kv26

In [38]:
env_frames

{'global': {}}

In [34]:
import sys
from collections import deque

class Tracer:
    def __init__(self):
        self.linenos = deque(maxlen=4) # TODO: PLS REFER BACK!!!!! CHANGE maxlen TO 2 IN A PY FILE!!!!

    def trace(self, frame, event, arg):
        if event == 'line' and frame.f_code is not self.log.__code__:
            self.linenos.append(frame.f_lineno)
        return self.trace

    def log(self):
        #print(self.linenos)
        print(self.linenos[0])
        #print(self.linenos)

tracer = Tracer()
sys._getframe().f_trace = tracer.trace
sys.settrace(tracer.trace)

def f():
    if True:
        print(inspect.currentframe().f_lineno)
    else:
        pass
f()
tracer.log()

24
3502


# code live insertion

## definitions for handling code-insertion

In [ ]:
def environment_dict(func, start_dict = {}):
    frame=inspect.currentframe().f_back
    func.__frame = frame # location in which the function was defined (make sure this is synonymous from where it is called from?? prob not)
    return func

# TODO: do this for lambda as well
def handle_def(decorator, code_list):
    """this function recursively handles define statements within functions by adding a line at the end of the 
    def that adds some lines of code to the function before it returns."""
    # does this by finding a 'def' statement, then counting the whitespaces of each following line until one has less than or equal to the amount
    # the def func had. Then inserts the new decorator line right before that.
    func_name = None
    def_whitespace_n = 0
    i = 0
    # find the first define statement.
    while not func_name or i >= len(code_list):
        if code_list[i].lstrip(' ')[:4] == "def ":
            func_name = code_list[i].lstrip(' ')[4:].split('(', 1)[0]
            def_whitespace_n = len(code_list[i]) - len(code_list[i].lstrip(' '))
        i = i + 1
    # if something was in a define statement, handle that.
    if func_name:
        # make line that will come after the define statement's end
        new_line = ' '*def_whitespace_n + func_name + '=' + decorator.__code__.co_name + '(' + func_name + ')' + '\n'
        # 
        # TODO: check to make sure this check changes as code_list gets bigger
        #
        while i < len(code_list):
            # check to see if the amount of preceeding whitespace in the define statement is greater than or equal to the amount in the current line
            if def_whitespace_n >= (len(code_list[i]) - len(code_list[i].lstrip(' '))):
                code_list.insert(i, new_line)
                return code_list
            # check to see if a function was defined within the function (if so do recursion)
            elif code_list[i].lstrip(' ')[:4] == "def ":
                # replace that part of the code list with the new code list
                code_list[i - 1:] = handle_def(decorator, code_list[i - 1:]) # i - 1 so we include the def statement
            i = i + 1  
        code_list.append(new_line)  
    return code_list

## definition for A and testing block

In [ ]:
def A():
    x = 5
    print("x is ", str(x))
    print(inspect.currentframe().f_locals) # !! nestedcode, !!varnames, reference, dir, locals, f_back
    print(dill.detect.varnames(A)) # returns a tuple (local vars, local vars referrenced by nested functions) 
    ref = dill.pointers.reference(x)
    print(ref)
    return x
cat = A()

x is  5
{'x': 5, 'y': [7, 2, 1, 3]}
(('x', 'y', 'ref'), ())
4629236608


In [12]:
print("OUTPUT BEFORE:")
A()
new_code_list = inspect.getsourcelines(A)[0]
#new_code_list = lines #handle_def(save_frame, lines)
new_code = "" # new_code_list[0] 
new_code_list[1] = '    print("jadbjhwjd")\n'
#print(new_code_list)
i = 0
while i < len(new_code_list):
    new_code += new_code_list[i]
    #new_code += line + '\n'
    i += 1
#new_code += '\n'
print("new code")
###
#new_code = dill.source.getsource(A, "A")
print(new_code)
#myast = ast.parse(new_code)
#code_obj = compile(myast, 'mymodule', 'exec')
#code_obj = compile(new_code, '<string>', 'exec')
print("OUTPUT:")
exec("""
def A():
    print("hello")
y = 7
A()""")

OUTPUT BEFORE:


NameError: name 'A' is not defined

# Trash

In [ ]:
#THROWING SHIT AT THE WALL
x = 5
def ahhh(y):
    x = y + 4
    #print(dir())
    #print(inspect.trace())
    #print(inspect.stack())
    print(inspect.currentframe().f_back)
    print(inspect.currentframe())
    return 8
#print(globals())
#print(dir())
#print(locals())
#ahhh(8)
#traceback.extract_tb()
#frame_infos = inspect.trace()
#print(inspect.trace())
#ahhh(8)
#print(len(inspect.stack()))
#print(inspect.stack())
#print(inspect.currentframe())
#aaa = inspect.getframeinfo(inspect.currentframe())
#print(type(aaa))
#print(aaa)
#print(inspect.getinnerframes(aaa))
stack = inspect.stack()
outer_frames = inspect.getouterframes(inspect.currentframe())
inner_frames = [frame for frame in stack if frame not in outer_frames]
print(inner_frames)
#print(current_frame)

In [ ]:
import inspect
import copy
import types
import dill


#Define a function we want to modify:
def test():
    print("Here")

#Run the function to check output
print("\n\nRunning Function...")
test()
#>>> Here

#Get the source code for the test function:
testSource2 = inspect.getsourcelines(test)[0]#
testSource = dill.source.getsource(test)
print("\n\ntestSource:")
print(testSource)


#Take the inner part of the source code and modify it how we want:
newtestinnersource = ''
testSourceLines = testSource.split('\n')
print("lines")
print(testSourceLines)
linenumber = 0 
for line in testSourceLines:
    if (linenumber > 0):
        if (len(line[4:]) > 0):
            newtestinnersource += line[4:] + '\n'
    linenumber += 1
#newtestinnersource += 'print("Here2")\n'
newtestinnersource += 'return 2\n'
#print("\n\nnewtestinnersource")
print("new")
print(newtestinnersource)


#Re-assign the function's code to be a compiled version of the `innersource`
code_obj = compile(newtestinnersource, '<string>', 'exec')
test.__code__ = copy.deepcopy(code_obj)
print("\n\nRunning Modified Function...")
a = test() #<- NOW HAS MODIFIED SOURCE CODE, AND PERFORMS NEW TASK
#>>>Here
#>>>Here2
testSource2
test.__code__
print("a:")
print(a)


In [ ]:
x = 5
y = x
x = 8
print(y)

5


# typing casting to try to get a valid Traceback

In [ ]:
import types
import typing

In [ ]:
"""FrameInfo(frame=<frame at 0x106e498a0, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/310886205.py', line 19, code <module>>, filename='/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/310886205.py', lineno=19, function='<module>', code_context=['stack = inspect.stack()\n'], index=0, positions=Positions(lineno=19, end_lineno=19, col_offset=8, end_col_offset=23)),"""
test2 = typing.cast(types.TracebackType, inspect.getframeinfo(inspect.currentframe()))
test2.f_lineno = test2.tb_lineno
#test = types.TracebackType(test2, inspect.currentframe(), 0, 19)


AttributeError: 'Traceback' object has no attribute 'tb_lineno'

In [ ]:
type(test2)
inspect.getinnerframes(typing.cast(types.TracebackType, inspect.getframeinfo(inspect.currentframe())))

In [ ]:
traceback.format_tb(inspect.getframeinfo(inspect.currentframe()))

AttributeError: 'Traceback' object has no attribute 'tb_frame'